In [88]:
rm(list = ls())
setwd(rprojroot::find_rstudio_root_file())
source("base/Preprocess_data.R")
source("base/Regression_model.R")
source("base/Knn_model.R")
source("base/Export_data.R")
source("base/Scores.R")


In [89]:
### Forecasts
# input data

run_model <- function(datetime_initialisation,forecast_mode,export,...) {
  #catchment_code <- "5410002"
  #datetime_initialisation = lubridate::make_date(2016,6,1)
  #horizon = horizon_mode(window_method = "dynamic", month_start = 9, month_end = 3)
  #predictor_list <- c("tem_mean_6months")
  #remove_wys <- c(1990,1940,2013)
  #water_units = waterunits(q = "m^3/s", y = "GL")
  #forecast_mode <- "both"
  #data_location_paths = get_default_datasets_path(meteo = "ens30avg", hydro = "ERA5Ens_SKGE")

    #pre-processed data
    data_input = preprocess_data(
        datetime_initialisation,
        forecast_mode,...)
    # ensemble volume forecast
    data_fore = forecast_vol_ensemble(
      data_input = data_input,
      forecast_mode = forecast_mode)
    # # ensemble flow forecast
    #q_fore = run_q_forecast(
    #  data_input = data_input,
    #  data_fore = data_fore,
    #  forecast_mode = forecast_mode)
    #export results
    output = export_data(
      data_input = data_input,
      data_fore = data_fore,
      export = export)
  
  return(output)
  
}

In [90]:
#all available catchments
catchments_attributes_filename = "data_input/attributes/attributes_49catchments_ChileCentral.csv" 
cod_cuencas = read.csv(catchments_attributes_filename)$cod_cuenca
cod_cuencas

[1] 3414001 3421001 3430003 3820001 4302001 4311001 4313001 4320001 4501001
[10] 4503001 4511002 4513001 4515002 4522002 4531002 4703002 5100001 5101001
[19] 5200001 5401003 5410002 5411001 5414001 5707002 5710001 5721001 5722001
[28] 5722002 5730008 5741001 6003001 6008005 6013001 6027001 6028001 7103001
[37] 7104002 7112001 7115001 7317005 7321002 7330001 7350003 7354002 7355002
[46] 7358001 7381001 8104001 8106001

In [91]:
#months of initialisation
months_initialisation = seq(4,9)
months_initialisation

[1] 4 5 6 7 8 9

In [92]:
# test climate indices
# climate indices
climate_indices = c("MEIv2", "SOI", "ONI", "BIENSO")
months_horizon = seq(1,2,1)
climate_grid = expand.grid(climate_indices, months_horizon)
climate_predictors = paste(climate_grid[,1], "mean", paste0(climate_grid[,2],"months"), sep = "_")
climate_predictors


[1] "MEIv2_mean_1months"  "SOI_mean_1months"    "ONI_mean_1months"   
[4] "BIENSO_mean_1months" "MEIv2_mean_2months"  "SOI_mean_2months"   
[7] "ONI_mean_2months"    "BIENSO_mean_2months"

In [93]:
run_model(     
    catchment_code = sample(cod_cuencas,1),
    datetime_initialisation = lubridate::make_date(2016,sample(months_initialisation,1),1),
    horizon = horizon_mode(window_method = "static", month_start = 9, month_end = 3),
    predictor_list = sample(climate_predictors),
    remove_wys = seq(1950,1980),
    water_units = waterunits(q = "m^3/s", y = "GL"),
    forecast_mode = "cv",
    data_location_paths = get_default_datasets_path(meteo = "ens30avg", hydro = "ERA5Ens_SKGE"),
    export = 'scores')


rmse_det,r2_det,mae_obs,crps_ens,crpss_climatology
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
355.6217,0.003297531,255.0302,202.1045,0.2075273


In [ ]:



library(foreach)
library(doParallel)
registerDoParallel(cores = parallel::detectCores())

model <-
  foreach(month_initialisation=months_initialisation,.combine = "c") %:%
  foreach(climate_predictor = climate_predictors,.combine = "c") %:%
  foreach(catchment_code=cod_cuencas) %dopar% {

    run_model(
      catchment_code = catchment_code,
      month_initialisation = month_initialisation,
      horizon_month_start = "sep",
      horizon_month_end = "mar",
      horizon_strategy = "static",
      predictor_list = climate_predictor,
      wy_holdout = 2022,
      remove_wys = seq(1950,1980),
      units_q = "m3/s",
      units_y = "GL",
      mode = "cv",
      export = "scores"
    )
  } %>% purrr::transpose()

stopImplicitCluster()



df <- cbind(rbindlist(model$info),rbindlist(model$scores))
# modify names
df <- dplyr::rename(df,'predictor_name' = 'predictor_list_corrected')

# order columns
df$month_initialisation <- factor(df$month_initialisation,levels = months_initialisation)
df$predictor_name <- factor(df$predictor_name)
df$catchment_code <- as.numeric(df$catchment_code)

saveRDS(df,paste0("data_output/scores/RDS/model_results_",today(),".RDS"))

